In [1]:
# read txt document of entry points
# also iterate over all tif files (adult 3D mask images) in the same path and perform analysis 
# all analysis except column analysis 

In [2]:
#  input and save paths
data_path = '/home/tli_project/Desktop/Data/controls/adult_columns/'
save_path = '/home/tli_project/Desktop/Data/controls/adult_columns/adult_cols_occupancies/'

# px_resolution = (0.4, 0.076, 0.076) #z,y,x resolution
save_name = 'adult_T4_cols_occupancies_Nov_2022.csv'
mask_image = False # not used in the code

In [3]:
cols_names = ['0,0', '0,1', '1,1', '1,0', 
              '0,-1', '-1,-1', '-1,0', 
              '0,2', '0,3', '0,-2', '0,-3',
              '1,2', '1,3', '1,4',
              '1,-1', '1,-2', '1,-3', '1,-4',
              '2,0', '2,1', '2,2', '2,3', '2,4', '2,5',
              '2,-1', '2,-2', '2,-3', '2,-4',
              '-1,1', '-1,2', '-1,3', '-1,4',
              '-1,-2', '-1,-3', '-1,-4',
              '-2,0', '-2,1', '-2,2', '-2,3', '-2,4', 
              '-2,-1', '-2,-2', '-2,-3', '-2,-4',
              '3,0', '3,1', '3,2']

cols_px_vals = {col:i+1 for i, col in enumerate(cols_names)}

In [4]:
import os
from os.path import basename
import utils.analysis as analysis
import utils.datautils as datautils
import matplotlib.pyplot as plt
import tifffile as tif
import numpy as np
import re
from tqdm import tqdm
import pandas as pd
import math
from read_roi import read_roi_zip as col_zip
from scipy import ndimage, spatial

In [5]:
# getting list of folders in data_path, where each folder has files for one neuron
N_folders = [name for name in os.listdir(data_path) if os.path.isdir(os.path.join(data_path, name))]
N_folders = [f for f in N_folders if 'T4' in f]
N_folders = sorted(N_folders)

In [17]:
def get_cols_labeled_neuron(n_file, cols_px_vals):
    files_list = datautils.get_file_names(data_path+n_file, group_by='', 
                                        order=True, nested_files=False, 
                                        criteria='')
    neu_path = [f for f in files_list if '_mask' in f.lower()][0]
    neuron = tif.imread(neu_path)
    mask_path = [f for f in files_list if (f[-6] == '_') & ('.tif' in f)][0]
    mask = tif.imread(mask_path)
    neuron[mask==0] = 0
    neuron[neuron!=0] =1

    # openning the cols_zip file
    cols_path = [f for f in files_list if 'zip' in f.lower()][0]
    Cols = col_zip(cols_path)

        # get columns centroids (only three are needed but the code is simpler this way)
    cols_centroids = {}
    for col, val in Cols.items():
        if val['type'] == 'oval':
                x0 = val['left']+int(val['width']/2); a = int(val['width']/2)  # x center, half width                                       
                y0 = val['top']+int(val['height']/2); b = int(val['height']/2)  # y center, half height  
                cols_centroids[col] = (y0,x0,a,b) 
    
    #[N,2] array of centroids of all the columns
    cols_centroids_arr = np.array([(i[0],i[1]) for i in cols_centroids.values()])
    cols_names = list(cols_centroids.keys())
    # cols_px_vals = {col:i+1 for i, col in enumerate(cols_centroids.keys())}

    # get the mask of each col based on distances from centroids 
    neu_PC = np.argwhere(np.ones_like(neuron[0])) #getting a point cloud of all the possible px in 2D [Y,X] array
    cols_masks = np.zeros_like(neuron[0])
    for px in tqdm(neu_PC):
        dist, ind = spatial.KDTree(cols_centroids_arr).query(px)
        col = cols_names[ind]
        cols_masks[px[0],px[1]] = cols_px_vals[col]
    
    mask_4D = np.broadcast_to(cols_masks, neuron.shape)
    col_labeled_neuron = neuron.copy()
    col_labeled_neuron[col_labeled_neuron!=0] = 1
    col_labeled_neuron = col_labeled_neuron * mask_4D
    
    # saving the result labeled image
    col_labeled_neuron = col_labeled_neuron.astype('uint8')
    name = save_path+n_file+'_s'+basename(mask_path)[:-4]+'_cols_labeled.tif'
    tif.imwrite(name, col_labeled_neuron, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'ZYX'})

    # option for saving the mask
    name = save_path+n_file+'_s'+basename(mask_path)[:-4]+'_cols_mask.tif'
    tif.imwrite(name, cols_masks, imagej=True, resolution=(1./0.076, 1./0.076),
                metadata={'spacing': 0.4, 'unit': 'um', 'finterval': 1/10, 'axes':'YX'})

    return col_labeled_neuron, cols_masks

def calc_col_occupancy(N_folders, cols_px_vals):
    col_occ_df_cols = list(cols_px_vals.keys())
    col_occ_df_cols.insert(0,'subtype')
    col_occ_df_cols.insert(0,'neuron')
    col_occ_df = pd.DataFrame(columns=col_occ_df_cols)
    for i, n_file in tqdm(enumerate(N_folders)):
        result_line = {}
        # print(n_file)
        # mask_path = [f for f in n_file if (f[-6] == '_') & ('.tif' in f)][0]
        result_line['neruon'] = n_file.split(' ')[0]
        result_line['subtype'] = n_file[-2]
        print(result_line['neruon'], result_line['subtype'])
        col_labeled_neuron, cols_masks = get_cols_labeled_neuron(n_file, cols_px_vals)
        for col, val in cols_px_vals.items():
            result_line[col] = len(col_labeled_neuron[col_labeled_neuron == val])
        col_occ_df = col_occ_df.append(result_line, ignore_index=True)
        col_occ_df = col_occ_df.loc[:,col_occ_df.sum()!=0]
    col_occ_df.to_csv(save_path+n_file+'_cols_occupanies.csv')
    return col_occ_df

In [18]:
# for n_file in tqdm(N_folders):
    # col_labeled_neuron, cols_masks = get_cols_labeled_neuron(n_file, cols_px_vals)
col_occ_df = calc_col_occupancy(N_folders, cols_px_vals)

0it [00:00, ?it/s]

T4-11 B
first 5 files
['/home/tli_project/Desktop/Data/controls/adult_columns/T4-11 (SubtypeB)/T4_11_mask.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-11 (SubtypeB)/T4_11.csv', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-11 (SubtypeB)/T4-11_B.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-11 (SubtypeB)/T4-11.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-11 (SubtypeB)/RoiSet.zip']


100%|██████████| 1048576/1048576 [00:32<00:00, 32116.99it/s]
1it [00:33, 33.01s/it]

T4-12 C
first 5 files
['/home/tli_project/Desktop/Data/controls/adult_columns/T4-12 (SubtypeC)/T4_12_mask.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-12 (SubtypeC)/T4_12.csv', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-12 (SubtypeC)/T4-12_C.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-12 (SubtypeC)/T4-12.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-12 (SubtypeC)/RoiSet.zip']


100%|██████████| 1048576/1048576 [00:33<00:00, 31085.50it/s]
2it [01:07, 33.60s/it]

...

100%|██████████| 249919/249919 [00:07<00:00, 31609.16it/s]
26it [08:46, 11.69s/it]

T4-5 D
first 5 files
['/home/tli_project/Desktop/Data/controls/adult_columns/T4-5 (SubtypeD)/a_MAX_T4-26_x59y186.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-5 (SubtypeD)/T4_5_mask.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-5 (SubtypeD)/T4_5.csv', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-5 (SubtypeD)/T4-5.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-5 (SubtypeD)/T4-05_D.tif']


100%|██████████| 1048576/1048576 [00:33<00:00, 31316.22it/s]
27it [09:20, 18.34s/it]

T4-6 D
first 5 files
['/home/tli_project/Desktop/Data/controls/adult_columns/T4-6 (SubtypeD)/T4_6_mask.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-6 (SubtypeD)/T4_6.csv', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-6 (SubtypeD)/T4-6.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-6 (SubtypeD)/T4-06_D.tif', '/home/tli_project/Desktop/Data/controls/adult_columns/T4-6 (SubtypeD)/RoiSet.zip']


100%|██████████| 1048576/1048576 [00:33<00:00, 31451.19it/s]
28it [09:53, 21.21s/it]
